<a href="https://colab.research.google.com/github/chw8207/Transformer/blob/main/Q%26A.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers

In [ ]:
!pip install farm-haystack

In [ ]:
!pip install farm-haystack[colab,inference]

In [28]:
import tensorflow as tf
from transformers import pipeline, AutoModelForQuestionAnswering, AutoTokenizer

### GPU 설정

In [10]:
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0' :
    raise SystemError('GPU device not found')
print(f'Found GPU at: {device_name}')

Found GPU at: /device:GPU:0


In [11]:
from tensorflow.python.client import device_lib

print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 11420901334832857200
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 15208677376
locality {
  bus_id: 1
  links {
  }
}
incarnation: 5765050721822656387
physical_device_desc: "device: 0, name: Tesla V100-SXM2-16GB, pci bus id: 0000:00:04.0, compute capability: 7.0"
xla_global_id: 416903419
]


### 방법론

### 방법0: 시행착오

In [12]:
# 디폴트 모델과 토크나이저
nlp_qa = pipeline('question-answering')

No model was supplied, defaulted to distilbert-base-cased-distilled-squad and revision 626af31 (https://huggingface.co/distilbert-base-cased-distilled-squad).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [13]:
sequence = """The traffic began to slow down on Pioneer Boulevard in Los Angeles,
making it difficult to get out of the city. However, WBGO was playing some cool jazz,
and the weather was cool, making it rather pleasant to be making it out of the city
on this Friday afternoon. Nat King Cole was singing as Jo and Maria slowly made
their way out of LA and drove toward Barstow. They planned to get to Las Vegas
early enough in the evening to have a nice dinner and go see a show.
"""


In [14]:
nlp_qa(context=sequence, question='Where is Pioneer Boulevard?')

{'score': 0.9879735112190247, 'start': 55, 'end': 66, 'answer': 'Los Angeles'}

#### 방법 1: NER firts

In [15]:
# 질문을 찾기 위해 NER 사용하기
nlp_ner = pipeline('ner')
print(nlp_ner(sequence))

No model was supplied, defaulted to dbmdz/bert-large-cased-finetuned-conll03-english and revision f2482bf (https://huggingface.co/dbmdz/bert-large-cased-finetuned-conll03-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


[{'entity': 'I-LOC', 'score': 0.9735258, 'index': 8, 'word': 'Pioneer', 'start': 34, 'end': 41}, {'entity': 'I-LOC', 'score': 0.9944825, 'index': 9, 'word': 'Boulevard', 'start': 42, 'end': 51}, {'entity': 'I-LOC', 'score': 0.9995776, 'index': 11, 'word': 'Los', 'start': 55, 'end': 58}, {'entity': 'I-LOC', 'score': 0.99956936, 'index': 12, 'word': 'Angeles', 'start': 59, 'end': 66}, {'entity': 'I-ORG', 'score': 0.9919844, 'index': 26, 'word': 'W', 'start': 121, 'end': 122}, {'entity': 'I-ORG', 'score': 0.9907504, 'index': 27, 'word': '##B', 'start': 122, 'end': 123}, {'entity': 'I-ORG', 'score': 0.9884582, 'index': 28, 'word': '##G', 'start': 123, 'end': 124}, {'entity': 'I-ORG', 'score': 0.9722683, 'index': 29, 'word': '##O', 'start': 124, 'end': 125}, {'entity': 'I-PER', 'score': 0.99668807, 'index': 59, 'word': 'Nat', 'start': 264, 'end': 267}, {'entity': 'I-PER', 'score': 0.9976484, 'index': 60, 'word': 'King', 'start': 268, 'end': 272}, {'entity': 'I-PER', 'score': 0.9986173, 'ind

In [16]:
# 위치 엔터티 질문
nlp_qa = pipeline('question-answering')
print('Question 1.', nlp_qa(context=sequence, question='Where is Pioneer Boulevard?'))
print('Question 2.', nlp_qa(context=sequence, question='Where is Los Angeles located?'))
print('Question 3.', nlp_qa(context=sequence, question='Where is LA?'))
print('Question 4.', nlp_qa(context=sequence, question='Where is Barstow?'))
print('Question 5.', nlp_qa(context=sequence, question='Where is Las Vegas located?'))

No model was supplied, defaulted to distilbert-base-cased-distilled-squad and revision 626af31 (https://huggingface.co/distilbert-base-cased-distilled-squad).
Using a pipeline without specifying a model name and revision in production is not recommended.


Question 1. {'score': 0.9879735112190247, 'start': 55, 'end': 66, 'answer': 'Los Angeles'}
Question 2. {'score': 0.9875389337539673, 'start': 34, 'end': 51, 'answer': 'Pioneer Boulevard'}
Question 3. {'score': 0.5090532302856445, 'start': 55, 'end': 66, 'answer': 'Los Angeles'}
Question 4. {'score': 0.36954542994499207, 'start': 387, 'end': 396, 'answer': 'Las Vegas'}
Question 5. {'score': 0.2183990627527237, 'start': 355, 'end': 362, 'answer': 'Barstow'}


In [17]:
# 개인 엔터티 질문
nlp_qa = pipeline('question-answering')
nlp_qa(context=sequence, question='Who was singing?')

No model was supplied, defaulted to distilbert-base-cased-distilled-squad and revision 626af31 (https://huggingface.co/distilbert-base-cased-distilled-squad).
Using a pipeline without specifying a model name and revision in production is not recommended.


{'score': 0.9653673768043518,
 'start': 264,
 'end': 277,
 'answer': 'Nat King Cole'}

In [18]:
nlp_qa(context=sequence, question='Who was going to Las Vegas?')

{'score': 0.4316229820251465,
 'start': 264,
 'end': 277,
 'answer': 'Nat King Cole'}

#### SRL first

In [19]:
nlp_qa(context=sequence, question='Who are they?')

{'score': 0.8486901521682739,
 'start': 293,
 'end': 305,
 'answer': 'Jo and Maria'}

In [20]:
nlp_qa(context=sequence, question='Who drove to Las Vegas?')

{'score': 0.35941487550735474,
 'start': 264,
 'end': 305,
 'answer': 'Nat King Cole was singing as Jo and Maria'}

#### ELECTRA를 사용한 질문-답변

In [21]:
print(nlp_qa.model)

DistilBertForQuestionAnswering(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
      

In [22]:
nlp_qa = pipeline('question-answering', model='google/electra-small-generator',
                  tokenizer='google/electra-small-generator')
nlp_qa(context=sequence, question='Who drove to Las Vegas?')

Some weights of ElectraForQuestionAnswering were not initialized from the model checkpoint at google/electra-small-generator and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


{'score': 0.0003573082503862679,
 'start': 206,
 'end': 237,
 'answer': 'to be making it out of the city'}

In [23]:
nlp_qa = pipeline('question-answering')
nlp_qa(context=sequence, question='What was slow?')

No model was supplied, defaulted to distilbert-base-cased-distilled-squad and revision 626af31 (https://huggingface.co/distilbert-base-cased-distilled-squad).
Using a pipeline without specifying a model name and revision in production is not recommended.


{'score': 0.46530431509017944, 'start': 0, 'end': 11, 'answer': 'The traffic'}

In [24]:
nlp_qa = pipeline('question-answering')
nlp_qa(context=sequence, question='What was playing?')

No model was supplied, defaulted to distilbert-base-cased-distilled-squad and revision 626af31 (https://huggingface.co/distilbert-base-cased-distilled-squad).
Using a pipeline without specifying a model name and revision in production is not recommended.


{'score': 0.48992660641670227, 'start': 143, 'end': 152, 'answer': 'cool jazz'}

In [25]:
nlp_qa = pipeline('question-answering')
nlp_qa(context=sequence, question='Who sees a show?')

No model was supplied, defaulted to distilbert-base-cased-distilled-squad and revision 626af31 (https://huggingface.co/distilbert-base-cased-distilled-squad).
Using a pipeline without specifying a model name and revision in production is not recommended.


{'score': 0.5588222146034241,
 'start': 264,
 'end': 277,
 'answer': 'Nat King Cole'}

#### RoBERTa 모델로 Haystack 탐구하기

In [26]:
# 트랜스포머에서 사용
model_name = 'deepset/roberta-base-squad2'

# 예측 얻기
nlp_qa = pipeline('question-answering', model=model_name, tokenizer=model_name)
QA_input = {
    'question': 'Where is Pioneer Boulevard located?',
    'context': sequence
}
res = nlp_qa(QA_input)

# 모델 로딩 % 토크나이징
model = AutoModelForQuestionAnswering.from_pretrained(model_name)
tokenizer = AutoModelForQuestionAnswering.from_pretrained(model_name)